In [8]:
# Allow importation of the others notebook
import import_ipynb

# pyTorch: module for the neural network
import torch
import torch.nn as nn

# load excel files 
# note 13 April 2021: the module might be deprecated => yes only for .xls 
#import xlrd - Using pylightxl instead
import pylightxl as xl

# time tools
from time import time, asctime

# module that list all files in a input directory
from glob import glob
 
# module that open/save data as csv
import csv

#import asyncio   # note 13/04/2021, this module was already anotated: might be useless

# Data Vizualisation module
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

#worm_segmentation imports. I cannot import worm_segmentation functions. This is a temporary solution.
#import numpy as np
#from skimage.morphology import skeletonize
#from PIL import Image 

# helper function 
# note 13/04/2021 instead of * each function should be load independently
from tools.Centriole_Characteristic import *
from tools.Extract_Experiment_Characteristic import *
from tools.CNN_Tools import *
from tools.Graphical_Tools import *
from tools.Centriole_Detection import *

#from tools.worm_segmentation import *

importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/Centriole_Characteristic.ipynb
importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/ToolBox.ipynb
importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/Midline_Edge_Reformater.ipynb
importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/Extract_Experiment_Characteristic.ipynb
importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/CNN_Tools.ipynb
importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/Graphical_Tools.ipynb
importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/Analysis_Tools.ipynb
importing Jupyter notebook from /home/macrostomum/git_environment/IJM-internship/tools/Centriole_Detection.ipynb


In [ ]:
def import_data(choice = int(input('1 = analyse all data in a folder; 2 = analyse a specific file.'))):
    files = [] 
    tif_list = []
    xls_list = []
    
    if choice == 1:
        tif_list = glob('./to_analyse/*.tif')
        xls_list = glob('./to_analyse/*.xlsm')
        
    elif choice == 2:
        file_name = input('Paste file name to be analysed, without extensions: ')
        tif_list = ['./to_analyse/' + file_name + '.tif']          
        xls_list = ['./to_analyse/' + file_name + '.xlsm']
        
    else:
        print('Option provided was not understood. Try again.')
        
        
    files = list(map(list, zip(xls_list, tif_list)))
    print('files to analyse: ', files)
    return files


files = import_data() #store function's return in a variable




def edge_and_midline_reformat(file):
    pathExcel = file[0]
    pathImg_100x = file[1]
    
    print(f"{asctime()}")
    print(f"Exl file loaded: {pathExcel}")
    print(f"Tif file loaded: {pathImg_100x}") 
    
    # Are you just testing the angle compensation?
    Test_Angle_Compensation = False

    #problem = 'classification'
    LOAD_MIDLINE_AND_EDGE_FROM_EXCEL = int(input('load midline and edge from excel? 1 = yes, 0 = no. ')) #useless line. Get from excel data

    if LOAD_MIDLINE_AND_EDGE_FROM_EXCEL == 1:
        db = get_xls_values(pathExcel)            #get_xls_values() opens excel file and loads the coordinates 
        x_mid = db['worm_midline']['x']
        y_mid = db['worm_midline']['y']
        x_edg = db['worm_edge']['x']
        y_edg = db['worm_edge']['y']
        
        # Invert the y axis for edge and midline
        newY_mid, newY_edg = [], [] 
        for y in y_mid:
            newY_mid.append(-y) 

        for y in y_edg: 
            newY_edg.append(-y) 

        # 'skeleton' of the worm
        worm = [x_mid, newY_mid, x_edg, newY_edg ]
        
    elif LOAD_MIDLINE_AND_EDGE_FROM_EXCEL == 0:   
        midline, edge = extract_worm_edge(path_img_10x, nBlur = 2, quantile = 0.01)
        worm = [midline[0], midline[1], edge[0], edge[1]]
    
    else:
        print('Option provided was not understood. Try again.')
    
    # Reformat midline and Edge in a given number of segment and sulbsegment
    midline_final = aggregate_segment_char(worm[0], worm[1], worm[2], worm[3], n_midline_seg = 50, n_sub_segment = 25, n_edge_seg = 200)

    print(f'{asctime()}: Edge and Midline reformated')
    
    return midline_final


midline_final = edge_and_midline_reformat(files[0])





def centriole_identification(pathImg_100x):# CENTRIOLES IDENTIFICATION
    
    # load the 100x image using OpenCV
    img = cv2.imread(pathImg_100x, cv2.IMREAD_UNCHANGED)
    print(f'{asctime()}: Image Loaded')

    # Compute differential of Gaussian then Otsu thresholding to try to get a mask of the centriole
    # return a binary image
    dog_img = dog_and_otsu(img)
    
    img_to_save = Image.fromarray((dog_img * 255).astype(np.uint8)) #updated this line. PIL pckg does not support [0-1] 
    newPath = pathImg_100x[:-4] + '_dog_otsu.tif'
    img_to_save.save(newPath)
        
    print(f'{asctime()}: Dog & Otsu computed ')

    
    # detect centriole using find_maxima algorithm 
    # note: find_maxima() is the implementation of the find_maxima module in imageJ
    # return a binary image
    find_maxima_img = find_maxima(img)
    
    img_to_save = Image.fromarray((find_maxima_img * 255).astype(np.uint8))  #updated this line
    newPath = pathImg_100x[:-4] + '_find_maxima.tif'
    img_to_save.save(newPath)

    print(f'{asctime()}: Find Maxima Computed')

    
    # combine the 2 previous approaches to get the final 
    # return a binary image
    combine_img = dog_img*find_maxima_img
    
    img_to_save = Image.fromarray((combine_img * 255).astype(np.uint8)) #updated this line
    newPath = pathImg_100x[:-4] + '_centriole_detected.tif'
    img_to_save.save(newPath)
    
    print(f'{asctime()}: Centriole detected')
    
    return combine_img, img

combine_img = centriole_identification(files[0][1])[0]
img = centriole_identification(files[0][1])[1]


#CENTRIOLE ANGLE PREDICTION FUNCTION
def centriole_angle_predictor(combine_img, img):
    
    if torch.cuda.is_available():                                  
        device = torch.device('cuda')
        print('Running on cuda')
    else:
        device = torch.device('cpu')
        print('Running on cpu')

    # Loading the Neural Network (VGG_Schmidtea) and the Weight of the network ()
    #13/04/2021: only classification is working
    
    #if problem == 'classification':
    model = VGG_Schmidtea(n_classes = 72).to(device)
    model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_classification.pth', map_location = device))
   # else:
       # model = VGG_Schmidtea(n_classes = 1).to(device)
       # model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_regression.pth', map_location = device))

    # Put the model in evaluation/prediction mode
    model.eval()

    # Extraction of coordinates where centrioles are
    ypts, xpts = np.where(combine_img == 1)

    a_list_of_centriole = []

    x_shape, y_shape = img.shape[1], img.shape[0] 
    xlim, ylim  = x_shape - 16 , y_shape - 16
    for i in range(len(ypts)):
        x, y = xpts[i], ypts[i]

        if y > 16 and x > 16 and y < ylim and x < xlim:
            #centriole_extracted = img.crop((xpts[i], ypts[i], xpts[i] + 32, ypts[i] + 32))
            centriole = img[y-16:y+16, x-16:x+16]
            centriole = np.asarray(centriole, dtype = "uint8")
            centriole = centriole.reshape(1 , 1, 32, 32)
            # Inside predictor:
            centriole = torch.from_numpy(centriole)
            centriole = centriole.float().to(device)

            with torch.no_grad():
                output = model(centriole)

            angle = output.max(1)[1]
            angle = angle.numpy()

            #print(centriole_extracted)
            #angle = predictor(model, centriole_extracted, device, problem = 'classification')
            a_list_of_centriole.append(((xpts[i], ypts[i]), angle[0]))
    
    print(f'{asctime()}: Centrioles angle predicted and compensated')
    
    return a_list_of_centriole

a_list_of_centriole = centriole_angle_predictor(combine_img, img)


def centriole_reposition_reorientation(a_list_of_centriole, pathImg_100x):

    shiftX = db['image_shift']['x']
    shiftY = db['image_shift']['y']

    shifted_centriole_list = []

    for a_centriole in a_list_of_centriole:
        xShifted = a_centriole[0][0] + shiftX
        yShifted = a_centriole[0][1] + shiftY
        #if problem == 'classification':
        shifted_centriole_list.append(((xShifted, -yShifted),a_centriole[1]*5+2.5))
       # else:
            #shifted_centriole_list.append(((xShifted, -yShifted),a_centriole[1]))


    reoriented_centriole = []
    for a_centriole in shifted_centriole_list:
        tmp_list = list(centriole_characterizator(a_centriole, midline_final))
        if db['worm_orientation'] == 'gauche' or db['worm_orientation'] == 'left':
            tmp_list[-2] = 1 - tmp_list[-2]
            tmp_list[-1] = math.degrees(math.atan2(-math.sin(math.radians(tmp_list[-1])), -math.cos(math.radians(tmp_list[-1]))))
        tmp_list.insert(1,a_centriole[0][0])
        tmp_list.insert(2,a_centriole[0][1])

        reoriented_centriole.append(tmp_list)


        # Save
    newPath = pathImg_100x[:-4] + '_DATA.csv'
    with open(newPath, 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(reoriented_centriole)

    print(f'{asctime()}: Centriole Dataset reformated')
    
    return reoriented_centriole

centriole_reposition_reorientation(a_list_of_centriole, files[0][1])


for i in range(5):
    print_a_antero_posterior_result(reoriented_centriole, i, n_ante_post_segment = 5, a_lat_size = 0.1, a_lat_step = 0.05, save = True, path = files[0][1][:-4])



1 = analyse all data in a folder; 2 = analyse a specific file.2
Paste file name to be analysed, without extensions: MAX_Mac_WT_Rootletin-G-02
files to analyse:  [['./to_analyse/MAX_Mac_WT_Rootletin-G-02.xlsm', './to_analyse/MAX_Mac_WT_Rootletin-G-02.tif']]
Tue May  4 10:41:24 2021
Exl file loaded: ./to_analyse/MAX_Mac_WT_Rootletin-G-02.xlsm
Tif file loaded: ./to_analyse/MAX_Mac_WT_Rootletin-G-02.tif
load midline and edge from excel? 1 = yes, 0 = no. 1
Tue May  4 10:41:32 2021: Edge and Midline reformated
Tue May  4 10:41:32 2021: Image Loaded
Tue May  4 10:41:34 2021: Differential of Gaussian performed
